<a href="https://colab.research.google.com/github/praffuln/agentic-ai/blob/master/langgraph_agentic_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: print hello world

print("hello world")


hello world


In [ ]:
!pip install google-generativeai tiktoken chromadb langgraph langchain langchain_community langchainhub ipykernel langchain_groq sentence_transformers


In [14]:
from typing import Annotated, Literal, Sequence, TypedDict
from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode

## Setup Keys For GOOGLE_API_KEY


In [ ]:

# Assigning value to variable
GEMINI_API_KEY=''
SERPER_API_KEY = ''
GROQ_API_KEY = ''


# setup environments


In [ ]:
import os
os.environ['SERPER_API_KEY'] = SERPER_API_KEY
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY
os.environ['GROQ_API_KEY'] = GROQ_API_KEY




## setup LLM

In [ ]:
from crewai import Agent, Task, Process, Crew, LLM

In [ ]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("Explain how AI works")
print(response.text)


In [ ]:
llm = LLM(
    model="gemini/gemini-1.5-flash",
    temperature=0.7
)

In [ ]:
from langchain_groq import ChatGroq
from sentence_transformers import SentenceTransformer # Import SentenceTransformer instead of GroqEmbedding


llm = ChatGroq(model="mixtral-8x7b-32768")

llm.invoke("tell me something about ai")

# Now use SentenceTransformer for embedding
embedding_model = SentenceTransformer('all-mpnet-base-v2')  # Choose a suitable pre-trained model
embedding = embedding_model.encode("tell me something about ai")

print(embedding) # Print the embedding to see the result

## Load Data

In [ ]:
urls= ["https://lilianweng.github.io/posts/2023-06-23-agent/",
      "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/"]
docs = [WebBaseLoader(url).load() for url in urls]

docs[0][0].metadata

In [ ]:
docs_list = [item for sublist in docs for item in sublist]

text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=100, chunk_overlap=5)
doc_splits=text_splitter.split_documents(docs_list)
doc_splits

In [42]:

from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")


vectorstore=Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chrome",
    embedding=embedding_model
)


<ipython-input-42-84e30c1413db>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")


In [43]:
retriever=vectorstore.as_retriever()


## Fuctions & Graph

In [16]:
def ai_assistant(state):
  pass
    # print("---CALL AGENT---")
    # messages = state['messages']
    # print(f"this is my message: {messages}")

    # if len(messages)>1:
    #     response=llm.invoke(messages[-1].content)
    #     return {"messages": [response]}
    # else:
    #     llm_with_tool = llm.bind_tools(tools)
    #     response = llm_with_tool.invoke(messages)
    #     #response=handle_query(messages)
    #     return {"messages": [response]}


In [17]:
def retrieve(state):
  pass

In [18]:
def rewrite(state):
  pass

In [19]:
def generate(state):
  pass

In [20]:
class AgentState(BaseModel):
    pass

In [21]:
workflow=StateGraph(AgentState)
workflow.add_node("My_AI_Assistant",ai_assistant)
workflow.add_node("Vector_Retriever", retrieve)
workflow.add_node("Query_Rewriter", rewrite)
workflow.add_node("Output_Generator", generate)


In [23]:
workflow.add_edge(START, "My_AI_Assistant")
workflow.add_edge("My_AI_Assistant", "Vector_Retriever")
# workflow.add_conditional_edges()

In [24]:
app = workflow.compile()

In [ ]:
app.invoke({"messages": [HumanMessage(content="tell me something about ai")]})